In [1]:
suppressPackageStartupMessages({
    library(org.Hs.eg.db)
    library(org.Mm.eg.db)
    library(org.Ggallus.eg.db)
    library(clusterProfiler)
    library(enrichplot)
    library(ggplot2)
})
options(enrichplot.colours = c("red","blue"))

Warning message:
“package ‘Biobase’ was built under R version 4.3.3”
Warning message:
“package ‘IRanges’ was built under R version 4.3.3”
Warning message:
“package ‘S4Vectors’ was built under R version 4.3.3”


In [98]:
Hsap2Mmus <- read.delim('../../../../01.data/orthologs_RBH/RBH.Hsap.Mmus.tsv')
head(Hsap2Mmus)

Hsap            Mmus              
1 ENSG00000221813 ENSMUSG00000049168
2 ENSG00000186440 ENSMUSG00000051509
3 ENSG00000197532 ENSMUSG00000066671
4 ENSG00000197706 ENSMUSG00000058251
5 ENSG00000187857 ENSMUSG00000044025
6 ENSG00000221931 ENSMUSG00000051095

In [2]:
Hsap_bg <- read.table("/mnt/data01/yuanzhen/01.Vertebrate_cell_evo/01.data/02.atlas/v_adata/Hsap_pep.ID")$V1
Mmus_bg <- read.table("/mnt/data01/yuanzhen/01.Vertebrate_cell_evo/01.data/02.atlas/v_adata/Mmus_pep.ID")$V1
Ggal_bg <- read.table('../2.bg/chicken.bg')$V1

In [3]:
Hsap_markers <- read.delim('../1.markers/human.ex.DEGs.subnuclei_class.EnsemblID.txt')
Mmus_markers <- read.delim('../1.markers/mouse.ex.DEGs.subnuclei_class.EnsemblID.txt')
Ggal_markers <- read.delim('../1.markers/chicken.ex.DEGs.subnuclei_class.EnsemblID.txt')

In [25]:
# GO for mouse
Mmis_ego2 <- enrichGO(gene         = unique(Mmus_markers$gene),
                 OrgDb         = org.Mm.eg.db,
                 keyType = "ENSEMBL",
                 pAdjustMethod = "BH",
                 pvalueCutoff = 0.05,
                 universe = Mmus_bg,
                 ont  = "BP")
Mmus_ego2 <- simplify(Mmis_ego2, cutoff=0.7, by="p.adjust", select_fun=min)


Hsap_ego2 <- enrichGO(gene         = unique(Hsap_markers$gene),
                 OrgDb         = org.Hs.eg.db,
                 keyType = "ENSEMBL",
                 pAdjustMethod = "BH",
                 pvalueCutoff = 0.05,
                 universe = Hsap_bg,
                 ont  = "BP")
Hsap_ego2 <- simplify(Hsap_ego2, cutoff=0.7, by="p.adjust", select_fun=min)

Ggal_ego2 <- enrichGO(gene         = unique(Ggal_markers$gene),
                 OrgDb         = org.Ggallus.eg.db,
                 keyType = "GID",
                 pAdjustMethod = "BH",
                 pvalueCutoff = 0.05,
                 universe = Ggal_bg,
                 ont  = "BP")
Ggal_ego2 <- simplify(Ggal_ego2, cutoff=0.7, by="p.adjust", select_fun=min)

In [121]:
Mmus_ego2@result$Description[grepl('ENSMUSG00000026686', Mmus_ego2@result$geneID)]


[1] "axonogenesis"                        
[2] "learning or memory"                  
[3] "cognition"                           
[4] "locomotory behavior"                 
[5] "regulation of neuron differentiation"
[6] "forebrain development"               
[7] "regulation of cell growth"

In [79]:
Hsap_ego2@result$Description[grepl('regulation of neuron projection development', Hsap_ego2@result$Description)]
Mmus_ego2@result$Description[grepl('neuron projection', Mmus_ego2@result$Description)]
Ggal_ego2@result$Description[grepl('develop', Ggal_ego2@result$Description)]

[1] "negative regulation of neuron projection development"

[1] "neuron projection organization"

[1] "regulation of neuron projection development"
[2] "regulation of nervous system development"

In [82]:
Hsap_ego2@result$Description[grepl('axon', Hsap_ego2@result$Description)]
Mmus_ego2@result$Description[grepl('axon', Mmus_ego2@result$Description)]
Ggal_ego2@result$Description[grepl('project', Ggal_ego2@result$Description)]

[1] "axon guidance"                       "negative regulation of axonogenesis"
[3] "axon regeneration"                   "response to axon injury"

[1] "axonogenesis"            "response to axon injury"

[1] "regulation of plasma membrane bounded cell projection organization"
[2] "regulation of neuron projection development"

In [80]:
Hsap_ego2@result$Description[grepl('migration', Hsap_ego2@result$Description)]
Mmus_ego2@result$Description[grepl('migration', Mmus_ego2@result$Description)]
Ggal_ego2@result$Description[grepl('migration', Ggal_ego2@result$Description)]

[1] "neuron migration"                                
[2] "neural crest cell migration"                     
[3] "mesenchymal cell migration"                      
[4] "vascular associated smooth muscle cell migration"

[1] "neuron migration"                                
[2] "negative regulation of cell migration"           
[3] "mesenchymal cell migration"                      
[4] "neural crest cell migration"                     
[5] "epithelium migration"                            
[6] "vascular associated smooth muscle cell migration"

[1] "regulation of epithelial cell migration"
[2] "epithelial cell migration"              
[3] "epithelium migration"

In [94]:
Hsap_ego2@result$Description[grepl('synaptic signaling', Hsap_ego2@result$Description)]
Mmus_ego2@result$Description[grepl('signal', Mmus_ego2@result$Description)]
Ggal_ego2@result$Description[grepl('synaptic signaling', Ggal_ego2@result$Description)]

[1] "regulation of trans-synaptic signaling"

[1] "glutamate receptor signaling pathway"                      
[2] "signal release from synapse"                               
[3] "retrograde trans-synaptic signaling"                       
[4] "ephrin receptor signaling pathway"                         
[5] "synaptic signaling via neuropeptide"                       
[6] "signaling receptor ligand precursor processing"            
[7] "positive regulation of calcium-mediated signaling"         
[8] "negative regulation of receptor signaling pathway via STAT"

[1] "synaptic signaling"                    
[2] "regulation of trans-synaptic signaling"

In [95]:
g1 <- unlist(strsplit(Hsap_ego2@result$geneID[which(Hsap_ego2@result$Description == 'regulation of trans-synaptic signaling')], split = '/'))
g2 <- unlist(strsplit(Mmus_ego2@result$geneID[which(Mmus_ego2@result$Description == 'signal release from synapse')], split = '/'))
g3 <- unlist(strsplit(Ggal_ego2@result$geneID[which(Ggal_ego2@result$Description == 'regulation of trans-synaptic signaling')], split = '/'))

In [114]:
Hsap_info[Hsap_info$EnsemblID %in% g1[g1 %in% Hsap2Mmus$Hsap] & Hsap_info$ohnologs,]

EnsemblID       Gene_name TFs   ohnologs
10   ENSG00000162631 NTNG1     FALSE TRUE    
22   ENSG00000122012 SV2C      FALSE TRUE    
37   ENSG00000183454 GRIN2A    FALSE TRUE    
50   ENSG00000196277 GRM7      FALSE TRUE    
52   ENSG00000120907 ADRA1A    FALSE TRUE    
76   ENSG00000118946 PCDH17    FALSE TRUE    
219  ENSG00000196358 NTNG2     FALSE TRUE    
290  ENSG00000117152 RGS4      FALSE TRUE    
296  ENSG00000127314 RAP1B     FALSE TRUE    
349  ENSG00000198400 NTRK1     FALSE TRUE    
493  ENSG00000106113 CRHR2     FALSE TRUE    
637  ENSG00000103496 STX4      FALSE TRUE    
729  ENSG00000184156 KCNQ3     FALSE TRUE    
735  ENSG00000198822 GRM3      FALSE TRUE    
742  ENSG00000163873 GRIK3     FALSE TRUE    
762  ENSG00000118432 CNR1      FALSE TRUE    
782  ENSG00000108947 EFNB3     FALSE TRUE    
797  ENSG00000117600 PLPPR4    FALSE TRUE    
818  ENSG00000115310 RTN4      FALSE TRUE    
820  ENSG00000161509 GRIN2C    FALSE TRUE    
832  ENSG00000139220 PPFIA2    FALSE TRUE    
860  ENSG00000162105 SHANK2    FALSE TRUE    
888  ENSG00000155511 GRIA1     FALSE TRUE    
919  ENSG00000131711 MAP1B     FALSE TRUE    
938  ENSG00000184985 SORCS2    FALSE TRUE    
1055 ENSG00000126583 PRKCG     FALSE TRUE    
1067 ENSG00000137766 UNC13C    FALSE TRUE    
1148 ENSG00000187323 DCC       FALSE TRUE    
1160 ENSG00000007516 BAIAP3    FALSE TRUE    
1171 ENSG00000164418 GRIK2     FALSE TRUE    
1195 ENSG00000118160 SLC8A2    FALSE TRUE    
1201 ENSG00000273079 GRIN2B    FALSE TRUE    
1252 ENSG00000075461 CACNG4    FALSE TRUE    
1284 ENSG00000138814 PPP3CA    FALSE TRUE    
1300 ENSG00000136099 PCDH8     FALSE TRUE    
1319 ENSG00000006116 CACNG3    FALSE TRUE    
1396 ENSG00000149403 GRIK4     FALSE TRUE    
1473 ENSG00000127946 HIP1      FALSE TRUE    
1480 ENSG00000106976 DNM1      FALSE TRUE    
1522 ENSG00000168309 FAM107A   FALSE TRUE    
1603 ENSG00000142408 CACNG8    FALSE TRUE    
1635 ENSG00000100266 PACSIN2   FALSE TRUE    
1646 ENSG00000146276 GABRR1    FALSE TRUE    
1659 ENSG00000188822 CNR2      FALSE TRUE    
1677 ENSG00000186716 BCR       FALSE TRUE    
1699 ENSG00000196549 MME       FALSE TRUE    
1786 ENSG00000105464 GRIN2D    FALSE TRUE    
1856 ENSG00000179603 GRM8      FALSE TRUE    
1863 ENSG00000081189 MEF2C      TRUE TRUE    
1868 ENSG00000148948 LRRC4C    FALSE TRUE    
1879 ENSG00000171532 NEUROD2    TRUE TRUE    
1950 ENSG00000154764 WNT7A     FALSE TRUE    
2003 ENSG00000138119 MYOF      FALSE TRUE    
2076 ENSG00000171132 PRKCE     FALSE TRUE    
2175 ENSG00000166862 CACNG2    FALSE TRUE    
2183 ENSG00000169071 ROR2      FALSE TRUE    
     family                                                                                                                                                                                                                                                                             
10   NTNG2,NTNG1                                                                                                                                                                                                                                                                        
22   SV2B,SV2A,SV2C                                                                                                                                                                                                                                                                     
37   GRIN2A,GRIN2C                                                                                                                                                                                                                                                                      
50   GRM6,GRM7,GRM2,GRM4,GRM3,GRM8                                                                                                                                                                                                                                 

In [104]:
Ggal_info[Ggal_info$Gene_name == 'Ntng2',]

EnsemblID Gene_name TFs ohnologs family

In [92]:
Mmus_info[Mmus_info$EnsemblID %in% g2,]

EnsemblID          Gene_name TFs   ohnologs
3    ENSMUSG00000039488 Cntn5     FALSE FALSE   
5    ENSMUSG00000027200 Sema6d    FALSE FALSE   
15   ENSMUSG00000060534 Dcc       FALSE  TRUE   
31   ENSMUSG00000028832 Stmn1     FALSE  TRUE   
50   ENSMUSG00000031673 Cdh11     FALSE  TRUE   
54   ENSMUSG00000030067 Foxp1      TRUE  TRUE   
55   ENSMUSG00000068748 Ptprz1    FALSE FALSE   
64   ENSMUSG00000032087 Dscaml1   FALSE FALSE   
87   ENSMUSG00000047261 Gap43     FALSE FALSE   
111  ENSMUSG00000006651 Aplp1     FALSE  TRUE   
129  ENSMUSG00000034684 Sema3f    FALSE FALSE   
143  ENSMUSG00000052920 Prkg1     FALSE FALSE   
145  ENSMUSG00000027797 Dclk1     FALSE FALSE   
149  ENSMUSG00000030077 Chl1      FALSE  TRUE   
162  ENSMUSG00000023175 Bsg       FALSE  TRUE   
171  ENSMUSG00000031137 Fgf13     FALSE  TRUE   
180  ENSMUSG00000044667 Plppr4    FALSE  TRUE   
181  ENSMUSG00000005583 Mef2c      TRUE  TRUE   
193  ENSMUSG00000008575 Nfib       TRUE  TRUE   
210  ENSMUSG00000032537 Ephb1     FALSE  TRUE   
241  ENSMUSG00000033174 Mgll      FALSE FALSE   
247  ENSMUSG00000045287 Rtn4rl1   FALSE FALSE   
248  ENSMUSG00000028289 Epha7     FALSE  TRUE   
259  ENSMUSG00000026686 Lmx1a      TRUE  TRUE   
263  ENSMUSG00000059921 Unc5c     FALSE  TRUE   
271  ENSMUSG00000028883 Sema3a    FALSE FALSE   
274  ENSMUSG00000055540 Epha6     FALSE  TRUE   
282  ENSMUSG00000040254 Sema3d    FALSE FALSE   
286  ENSMUSG00000029245 Epha5     FALSE  TRUE   
290  ENSMUSG00000022636 Alcam     FALSE FALSE   
⋮    ⋮                  ⋮         ⋮     ⋮       
773  ENSMUSG00000052516 Robo2     FALSE  TRUE   
783  ENSMUSG00000018698 Lhx1       TRUE FALSE   
794  ENSMUSG00000000861 Bcl11a     TRUE  TRUE   
804  ENSMUSG00000050965 Prkca     FALSE  TRUE   
814  ENSMUSG00000031558 Slit2     FALSE  TRUE   
829  ENSMUSG00000052504 Epha3     FALSE  TRUE   
834  ENSMUSG00000026872 Zeb2       TRUE  TRUE   
845  ENSMUSG00000028519 Dab1      FALSE FALSE   
863  ENSMUSG00000021068 Nin       FALSE  TRUE   
865  ENSMUSG00000027967 Neurog2    TRUE  TRUE   
873  ENSMUSG00000029697 Fezf1      TRUE FALSE   
883  ENSMUSG00000057329 Bcl2      FALSE FALSE   
908  ENSMUSG00000025885 Myo5b     FALSE  TRUE   
953  ENSMUSG00000035513 Ntng2     FALSE  TRUE   
958  ENSMUSG00000059857 Ntng1     FALSE  TRUE   
975  ENSMUSG00000015829 Tnr       FALSE FALSE   
982  ENSMUSG00000026235 Epha4     FALSE  TRUE   
997  ENSMUSG00000025969 Nrp2      FALSE FALSE   
1006 ENSMUSG00000024421 Lama3     FALSE  TRUE   
1038 ENSMUSG00000032128 Robo3     FALSE  TRUE   
1063 ENSMUSG00000000305 Cdh4      FALSE  TRUE   
1070 ENSMUSG00000056427 Slit3     FALSE  TRUE   
1085 ENSMUSG00000022054 Nefm      FALSE FALSE   
1089 ENSMUSG00000020396 Nefh      FALSE FALSE   
1092 ENSMUSG00000030092 Cntn6     FALSE FALSE   
1095 ENSMUSG00000048915 Efna5     FALSE FALSE   
1107 ENSMUSG00000032064 Dixdc1    FALSE  TRUE   
1114 ENSMUSG00000002489 Tiam1     FALSE FALSE   
1145 ENSMUSG00000038264 Sema7a    FALSE FALSE   
1160 ENSMUSG00000025582 Nptx1     FALSE FALSE   
     family                                                                              
3                                                                                        
5                                                                                        
15   Dcc,Neo1                                                                            
31   Stmn3,Stmn4,Stmn1,Stmn2                                                             
50   Cdh18,Cdh12,Cdh10,Cdh22,Cdh11,Cdh9,Cdh24,Cdh6,Cdh20,Cdh5,Cdh8                       
54   Foxp3,Foxp1,Foxp2                                                                   
55                                                                                       
64                                                                                       
87                                                                                       
111  App,Aplp2,Aplp1                                               

In [61]:
Mmus_info <- read.delim('/mnt/data01/yuanzhen/01.Vertebrate_cell_evo/13.case_analysis/1.cerebellum/6.SuppTables/SuppTable.cerebellar_subnuclei_markers_info.Mmus.txt')
Hsap_info <- read.delim('/mnt/data01/yuanzhen/01.Vertebrate_cell_evo/13.case_analysis/1.cerebellum/6.SuppTables/SuppTable.cerebellar_subnuclei_markers_info.Hsap.txt')
Ggal_info <- read.delim('/mnt/data01/yuanzhen/01.Vertebrate_cell_evo/13.case_analysis/1.cerebellum/6.SuppTables/SuppTable.cerebellar_subnuclei_markers_info.Ggal.txt')

In [38]:
Mmus_info[Mmus_info$EnsemblID %in% g1,]

EnsemblID          Gene_name TFs   ohnologs
3    ENSMUSG00000039488 Cntn5     FALSE FALSE   
5    ENSMUSG00000027200 Sema6d    FALSE FALSE   
15   ENSMUSG00000060534 Dcc       FALSE  TRUE   
31   ENSMUSG00000028832 Stmn1     FALSE  TRUE   
50   ENSMUSG00000031673 Cdh11     FALSE  TRUE   
54   ENSMUSG00000030067 Foxp1      TRUE  TRUE   
55   ENSMUSG00000068748 Ptprz1    FALSE FALSE   
64   ENSMUSG00000032087 Dscaml1   FALSE FALSE   
87   ENSMUSG00000047261 Gap43     FALSE FALSE   
111  ENSMUSG00000006651 Aplp1     FALSE  TRUE   
129  ENSMUSG00000034684 Sema3f    FALSE FALSE   
143  ENSMUSG00000052920 Prkg1     FALSE FALSE   
145  ENSMUSG00000027797 Dclk1     FALSE FALSE   
149  ENSMUSG00000030077 Chl1      FALSE  TRUE   
162  ENSMUSG00000023175 Bsg       FALSE  TRUE   
171  ENSMUSG00000031137 Fgf13     FALSE  TRUE   
180  ENSMUSG00000044667 Plppr4    FALSE  TRUE   
181  ENSMUSG00000005583 Mef2c      TRUE  TRUE   
193  ENSMUSG00000008575 Nfib       TRUE  TRUE   
210  ENSMUSG00000032537 Ephb1     FALSE  TRUE   
241  ENSMUSG00000033174 Mgll      FALSE FALSE   
247  ENSMUSG00000045287 Rtn4rl1   FALSE FALSE   
248  ENSMUSG00000028289 Epha7     FALSE  TRUE   
259  ENSMUSG00000026686 Lmx1a      TRUE  TRUE   
263  ENSMUSG00000059921 Unc5c     FALSE  TRUE   
271  ENSMUSG00000028883 Sema3a    FALSE FALSE   
274  ENSMUSG00000055540 Epha6     FALSE  TRUE   
282  ENSMUSG00000040254 Sema3d    FALSE FALSE   
286  ENSMUSG00000029245 Epha5     FALSE  TRUE   
290  ENSMUSG00000022636 Alcam     FALSE FALSE   
⋮    ⋮                  ⋮         ⋮     ⋮       
773  ENSMUSG00000052516 Robo2     FALSE  TRUE   
783  ENSMUSG00000018698 Lhx1       TRUE FALSE   
794  ENSMUSG00000000861 Bcl11a     TRUE  TRUE   
804  ENSMUSG00000050965 Prkca     FALSE  TRUE   
814  ENSMUSG00000031558 Slit2     FALSE  TRUE   
829  ENSMUSG00000052504 Epha3     FALSE  TRUE   
834  ENSMUSG00000026872 Zeb2       TRUE  TRUE   
845  ENSMUSG00000028519 Dab1      FALSE FALSE   
863  ENSMUSG00000021068 Nin       FALSE  TRUE   
865  ENSMUSG00000027967 Neurog2    TRUE  TRUE   
873  ENSMUSG00000029697 Fezf1      TRUE FALSE   
883  ENSMUSG00000057329 Bcl2      FALSE FALSE   
908  ENSMUSG00000025885 Myo5b     FALSE  TRUE   
953  ENSMUSG00000035513 Ntng2     FALSE  TRUE   
958  ENSMUSG00000059857 Ntng1     FALSE  TRUE   
975  ENSMUSG00000015829 Tnr       FALSE FALSE   
982  ENSMUSG00000026235 Epha4     FALSE  TRUE   
997  ENSMUSG00000025969 Nrp2      FALSE FALSE   
1006 ENSMUSG00000024421 Lama3     FALSE  TRUE   
1038 ENSMUSG00000032128 Robo3     FALSE  TRUE   
1063 ENSMUSG00000000305 Cdh4      FALSE  TRUE   
1070 ENSMUSG00000056427 Slit3     FALSE  TRUE   
1085 ENSMUSG00000022054 Nefm      FALSE FALSE   
1089 ENSMUSG00000020396 Nefh      FALSE FALSE   
1092 ENSMUSG00000030092 Cntn6     FALSE FALSE   
1095 ENSMUSG00000048915 Efna5     FALSE FALSE   
1107 ENSMUSG00000032064 Dixdc1    FALSE  TRUE   
1114 ENSMUSG00000002489 Tiam1     FALSE FALSE   
1145 ENSMUSG00000038264 Sema7a    FALSE FALSE   
1160 ENSMUSG00000025582 Nptx1     FALSE FALSE   
     family                                                                              
3                                                                                        
5                                                                                        
15   Dcc,Neo1                                                                            
31   Stmn3,Stmn4,Stmn1,Stmn2                                                             
50   Cdh18,Cdh12,Cdh10,Cdh22,Cdh11,Cdh9,Cdh24,Cdh6,Cdh20,Cdh5,Cdh8                       
54   Foxp3,Foxp1,Foxp2                                                                   
55                                                                                       
64                                                                                       
87                                                                                       
111  App,Aplp2,Aplp1                                               